In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drop_cols = [col for col in ["TX_DATETIME", "TRANSACTION_ID"] if col in df.columns]
df = df.drop(columns=drop_cols)


In [ ]:
from sklearn.preprocessing import LabelEncoder

le_customer = LabelEncoder()
le_terminal = LabelEncoder()

df["CUSTOMER_ID"] = le_customer.fit_transform(df["CUSTOMER_ID"])
df["TERMINAL_ID"] = le_terminal.fit_transform(df["TERMINAL_ID"])

if "TX_DATETIME" in df.columns:
    df["TX_DATETIME"] = pd.to_datetime(df["TX_DATETIME"])
    df["TX_YEAR"] = df["TX_DATETIME"].dt.year
    df["TX_MONTH"] = df["TX_DATETIME"].dt.month
    df["TX_DAY"] = df["TX_DATETIME"].dt.day
    df["TX_HOUR"] = df["TX_DATETIME"].dt.hour
    df["TX_MINUTE"] = df["TX_DATETIME"].dt.minute
    df["TX_SECOND"] = df["TX_DATETIME"].dt.second

    # Drop original datetime column
    df = df.drop(columns=["TX_DATETIME"])

print("✅ Final dataset shape:", df.shape)
print(df.head())


✅ Final dataset shape: (50068, 17)
   CUSTOMER_ID  TERMINAL_ID  TX_AMOUNT  TX_FRAUD     TX_DATE  TX_HOUR  TX_DAY  \
0          202          181      16.04         0  2024-01-01        0       1   
1         1182          121      44.31         0  2024-01-01        0       1   
2         1521          211      25.64         0  2024-01-01        0       1   
3         1489          361      32.31         0  2024-01-01        0       1   
4          755          587      14.84         0  2024-01-01        0       1   

   TX_WEEKDAY  is_high_amount  customer_avg_amount  amount_vs_customer_avg  \
0           0               0            36.162857                0.431614   
1           0               0            30.506364                1.406383   
2           0               0            34.810000                0.716001   
3           0               0            27.788235                1.122333   
4           0               0            21.458333                0.660779   

   termin

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pickle

df = pd.read_csv("/content/drive/MyDrive/fraud_dataset.csv")

print("✅ Dataset loaded. Shape:", df.shape)
print(df.head())

le_customer = LabelEncoder()
le_terminal = LabelEncoder()

if "CUSTOMER_ID" in df.columns:
    df["CUSTOMER_ID"] = le_customer.fit_transform(df["CUSTOMER_ID"])

if "TERMINAL_ID" in df.columns:
    df["TERMINAL_ID"] = le_terminal.fit_transform(df["TERMINAL_ID"])

if "TX_DATETIME" in df.columns:
    df["TX_DATETIME"] = pd.to_datetime(df["TX_DATETIME"])
    df["TX_YEAR"] = df["TX_DATETIME"].dt.year
    df["TX_MONTH"] = df["TX_DATETIME"].dt.month
    df["TX_DAY"] = df["TX_DATETIME"].dt.day
    df["TX_HOUR"] = df["TX_DATETIME"].dt.hour
    df["TX_MINUTE"] = df["TX_DATETIME"].dt.minute
    df["TX_SECOND"] = df["TX_DATETIME"].dt.second

    # Drop original datetime column
    df = df.drop(columns=["TX_DATETIME"])

if "TRANSACTION_ID" in df.columns:
    df = df.drop(columns=["TRANSACTION_ID"])

print("✅ After preprocessing, shape:", df.shape)

X = df.drop(columns=["TX_FRAUD"])   # Features
y = df["TX_FRAUD"]                  # Target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    class_weight="balanced",
    n_jobs=-1
)

model.fit(X_train, y_train)

print("✅ Model training complete.")

with open("fraud_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("customer_encoder.pkl", "wb") as f:
    pickle.dump(le_customer, f)

with open("terminal_encoder.pkl", "wb") as f:
    pickle.dump(le_terminal, f)

print("🎉 Model and encoders saved successfully!")


✅ Dataset loaded. Shape: (50068, 6)
   TRANSACTION_ID          TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
0             268  2024-01-01 00:00:44      C00203      T00182      16.04   
1             102  2024-01-01 00:02:23      C01183      T00122      44.31   
2             130  2024-01-01 00:05:46      C01522      T00212      25.64   
3             334  2024-01-01 00:07:52      C01490      T00362      32.31   
4             285  2024-01-01 00:09:18      C00756      T00588      14.84   

   TX_FRAUD  
0         0  
1         0  
2         0  
3         0  
4         0  
✅ After preprocessing, shape: (50068, 10)
Train size: (35047, 9) Test size: (15021, 9)
✅ Model training complete.
🎉 Model and encoders saved successfully!


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoders
customer_encoder = LabelEncoder()
terminal_encoder = LabelEncoder()

# Fit and transform
df["CUSTOMER_ID"] = customer_encoder.fit_transform(df["CUSTOMER_ID"])
df["TERMINAL_ID"] = terminal_encoder.fit_transform(df["TERMINAL_ID"])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=["TX_FRAUD"])
y = df["TX_FRAUD"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
import pickle

# Save the trained model
with open("fraud_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save encoders
with open("customer_encoder.pkl", "wb") as f:
    pickle.dump(customer_encoder, f)

with open("terminal_encoder.pkl", "wb") as f:
    pickle.dump(terminal_encoder, f)

print("✅ Model and encoders saved successfully!")


✅ Model and encoders saved successfully!
